In [76]:

import pandas
from keras.models import Sequential
from keras.layers import Dense

# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [77]:

dataframe = pandas.read_csv("persistence/training.csv")
dataset = dataframe.drop('awardedMatches', axis=1).drop('id', axis=1).values
X = dataset[:,0:98].astype(float)
y = dataset[:, 98:].astype(int)
print(X.shape, y.shape)

(230, 98) (230, 3)


In [78]:

# define the model
model = Sequential()
model.add(Dense(90, input_dim=98, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
23/23 [==============================] - 1s 3ms/step - loss: 147.2495
Epoch 2/150
23/23 [==============================] - 0s 8ms/step - loss: 45.4573
Epoch 3/150
23/23 [==============================] - 0s 6ms/step - loss: 23.9743
Epoch 4/150
23/23 [==============================] - 0s 6ms/step - loss: 18.3069

In [79]:

...
results = list()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# define evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# enumerate folds
for train_ix, test_ix in cv.split(X):
	# prepare data
	X_train, X_test = X[train_ix], X[test_ix]
	y_train, y_test = y[train_ix], y[test_ix]
	st_x= MinMaxScaler()
	X_train= st_x.fit_transform(X_train)
	X_test= st_x.transform(X_test)
	# fit model
	model.fit(X_train, y_train, verbose=0, epochs=100)
	# make a prediction on the test set
	yhat = model.predict(X_test)
	# print(yhat)
	# print(y_test)
	# round probabilities to class labels
	yhat = yhat.round()
	# calculate accuracy
	acc = accuracy_score(y_test, yhat)
	# store result
	print('>%.2f' % acc)
	results.append(acc)

>0.35
>0.17
>0.22
>0.17
>0.30
>0.30
>0.43
>0.52
>0.43
>0.43
>0.43
>0.39
>0.52
>0.61
>0.74
>0.78
>0.83
>0.74
>0.78
>0.78
>0.91
>0.91
>0.96
>0.96
>1.00
>0.91
>0.96
>1.00
>1.00
>1.00


In [80]:
print('Average accuracy: %.2f%% \nStandart deviation: %.2f%%' % ((mean(results))*100, (std(results)*100)))

Average accuracy: 65.22% 
Standart deviation: 27.91%
